In [1]:
import os
import sys
import pandas as pd
import numpy as np
from rpy2.robjects import packages, pandas2ri, r
import dask.dataframe as dd

PROJECT_PATH = os.path.abspath(os.path.join(os.getcwd(), '../'))

if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)
    
from src.data.fitzroy_data import fitzroy, r_to_pandas

In [2]:
# Player data matches have weird round labelling system (lots of strings for finals matches),
# so using round numbers from match_results
match_df = r_to_pandas(r('fitzRoy::match_results'))
player_df = (r_to_pandas(fitzroy().get_afltables_stats())
             # Some player data venues have trailing spaces
             .assign(venue=lambda x: x['venue'].str.strip())
             # Player data match IDs are wrong for recent years. The easiest way to add correct ones
             # is to graft on the IDs from match_results. Also, match_results round_numbers are more useful.
             .merge(match_df[['date', 'venue', 'round_number', 'game']], on=['date', 'venue'], how='left')
             # As of 11-10-2018, match_results is still missing finals data from 2018.
             # Joining on date/venue leaves two duplicates played at M.C.G. on 29-4-1986 & 9-8-1986,
             # but that's an acceptable loss of data and easier than munging team names
             .dropna()
             .rename(columns={'season': 'year',
                              'time_on_ground__': 'time_on_ground',
                              'id': 'player_id',
                              'game': 'match_id'})
             .astype({'year': int, 'match_id': int})
             .assign(player_name=lambda x: x['first_name'] + ' ' + x['surname'],
                     # Need to add year to ID, because there are some player_id/match_id combos, decades apart,
                     # that by chance overlap
                     id=lambda x: x['player_id'].astype(str) + x['match_id'].astype(str) + x['year'].astype(str))
             .drop(['first_name', 'surname', 'round', 'local_start_time', 'attendance', 'hq1g', 'hq1b',
                    'hq2g', 'hq2b', 'hq3g', 'hq3b', 'hq4g', 'hq4b', 'aq1g', 'aq1b', 'aq2g', 'aq2b', 'aq3g', 'aq3b',
                    'aq4g', 'aq4b', 'jumper_no_', 'umpire_1', 'umpire_2', 'umpire_3', 'umpire_4',
                    'substitute', 'group_id', 'date', 'venue'], axis=1)
             # Some early matches (1800s) have fully-duplicated rows
             .drop_duplicates()
             .set_index('id')
             .sort_index())

/usr/local/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)
/usr/local/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Returning data from 1897-01-01 to 2018-10-14

  warnings.warn(x, RRuntimeWarning)




|==                                                    |  4% ~4 s remaining

 

 

 

 

 


|===                                                   |  7% ~5 s remaining

 

 

 

 

 


/usr/local/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Downloading data


  warnings.warn(x, RRuntimeWarning)




|====                                                  |  9% ~5 s remaining

 

 

 

 

 


|======                                                | 11% ~5 s remaining

 

 

 

 

 


|=======                                               | 13% ~5 s remaining

 

 

 

 

 


|========                                              | 16% ~5 s remaining

 

 

 

 

 


|=========                                             | 18% ~5 s remaining

 

 

 

 

 


|==========                                            | 20% ~5 s remaining

 

 

 

 

 


|============                                          | 22% ~5 s remaining

 

 

 

 

 


|=============                                         | 24% ~5 s remaining

 

 

 

 

 


|==============                                        | 27% ~5 s remaining

 

 

 

 

 


|===============                                       | 29% ~5 s remaining

 

 

 

 

 


|================                                      | 31% ~4 s re

/usr/local/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 
Finished downloading data. Processing XMLs


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Warning:
  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  396 parsing failures.
row # A tibble: 5 x 5 col     row col   expected   actual file                                             expected   <int> <chr> <chr>      <chr>  <chr>                                            actual 1  8713 Round an integer QF     'https://afltables.com/afl/stats/2018_stats.txt' file 2  8714 Round an integer QF     'https://afltables.com/afl/stats/2018_stats.txt' row 3  8715 Round an integer QF     'https://afltables.com/afl/stats/2018_stats.txt' col 4  8716 Round an integer QF     'https://afltables.com/afl/stats/2018_stats.txt' expected 5  8717 Round an integer QF     '

In [3]:
STATS_COLS = ['player_id', 'kicks', 'marks', 'handballs', 'goals', 'behinds', 'hit_outs', 'tackles',
               'rebounds', 'inside_50s', 'clearances', 'clangers', 'frees_for',
               'frees_against', 'contested_possessions',
               'uncontested_possessions', 'contested_marks', 'marks_inside_50',
               'one_percenters', 'bounces', 'goal_assists', 'time_on_ground']
MATCH_COLS = ['year', 'home_team', 'home_score', 'away_team', 'away_score', 'round_number', 'match_id']

player_groups = player_df[STATS_COLS].groupby('player_id', group_keys=False)

rolling_stats = player_groups.rolling(window=23).mean()
expanding_stats = player_groups.expanding(1).mean()
expanding_rolling_stats = rolling_stats.fillna(expanding_stats).sort_index()

brownlow_last_year = (player_df[['player_id', 'year', 'brownlow_votes']]
                      .groupby(['player_id', 'year'], group_keys=True)
                      .sum()
                      # Grouping by player to shift by year
                      .groupby(level=0)
                      .shift()
                      .fillna(0)
                      .rename(columns={'brownlow_votes': 'last_year_brownlow_votes'}))
brownlow_df = (player_df[MATCH_COLS + ['player_id', 'playing_for', 'player_name']]
                .merge(brownlow_last_year, on=['player_id', 'year'], how='left')
                .set_index(player_df.index))

In [7]:
MATCH_STATS_COLS = ['at_home', 'match_id', 'score', 'oppo_score']

player_stats = (pd
                .concat([brownlow_df, expanding_rolling_stats], axis=1, sort=True)
                .assign(cum_games_played=player_groups.cumcount()))

home_stats = (player_stats
              [player_stats['playing_for'] == player_stats['home_team']]
              .drop(['playing_for'], axis=1)
              .rename(columns=lambda x: x.replace('home_', '').replace('away_', 'oppo_'))
              .assign(at_home=1))

away_stats = (player_stats
              [player_stats['playing_for'] == player_stats['away_team']]
              .drop(['playing_for'], axis=1)
              .rename(columns=lambda x: x.replace('away_', '').replace('home_', 'oppo_'))
              .assign(at_home=0))

# Filter out player_id & add brownlow stats
player_aggs = {col: 'sum' for col in STATS_COLS[1:] + ['last_year_brownlow_votes']}
# Since match stats are the same across player rows, taking the mean is the easiest way to aggregate them
match_aggs = {col: 'mean' for col in MATCH_STATS_COLS}

aggregations = {**player_aggs, **match_aggs}

team_df = (pd
           # Need to sort df columns, because pandas freaks out if columns are in different order
           .concat([home_stats[home_stats.columns.sort_values()],
                    away_stats[away_stats.columns.sort_values()]],
                   sort=True)
           .drop(['player_id', 'player_name'], axis=1)
           .groupby(['team', 'year', 'round_number', 'oppo_team'])
           .aggregate(aggregations)
           .reset_index()
           .set_index(['team', 'year', 'round_number'], drop=False))

team_df

team  year  round_number  \
team             year round_number                                         
Adelaide         1991 1.0                   Adelaide  1991           1.0   
                      2.0                   Adelaide  1991           2.0   
                      3.0                   Adelaide  1991           3.0   
                      4.0                   Adelaide  1991           4.0   
                      5.0                   Adelaide  1991           5.0   
                      6.0                   Adelaide  1991           6.0   
                      7.0                   Adelaide  1991           7.0   
                      9.0                   Adelaide  1991           9.0   
                      10.0                  Adelaide  1991          10.0   
                      11.0                  Adelaide  1991          11.0   
                      12.0                  Adelaide  1991          12.0   
                      13.0                  Adelaide  1991          13.0   
                      14.0                  Adelaide  1991          14.0   
                      15.0                  Adelaide  1991          15.0   
                      16.0                  Adelaide  1991          16.0   
                      17.0                  Adelaide  1991          17.0   
                      18.0                  Adelaide  1991          18.0   
                      19.0                  Adelaide  1991          19.0   
                      20.0                  Adelaide  1991          20.0   
                      21.0                  Adelaide  1991          21.0   
                      22.0                  Adelaide  1991          22.0   
                      24.0                  Adelaide  1991          24.0   
                 1992 1.0                   Adelaide  1992           1.0   
                      3.0                   Adelaide  1992           3.0   
                      4.0                   Adelaide  1992           4.0   
                      5.0                   Adelaide  1992           5.0   
                      6.0                   Adelaide  1992           6.0   
                      7.0                   Adelaide  1992           7.0   
                      8.0                   Adelaide  1992           8.0   
                      9.0                   Adelaide  1992           9.0   
...                                              ...   ...           ...   
Western Bulldogs 2017 16.0          Western Bulldogs  2017          16.0   
                      17.0          Western Bulldogs  2017          17.0   
                      18.0          Western Bulldogs  2017          18.0   
                      19.0          Western Bulldogs  2017          19.0   
                      20.0          Western Bulldogs  2017          20.0   
                      21.0          Western Bulldogs  2017          21.0   
                      22.0          Western Bulldogs  2017          22.0   
                      23.0          Western Bulldogs  2017          23.0   
                 2018 1.0           Western Bulldogs  2018           1.0   
                      2.0           Western Bulldogs  2018           2.0   
                      3.0           Western Bulldogs  2018           3.0   
                      4.0           Western Bulldogs  2018           4.0   
                      5.0           Western Bulldogs  2018           5.0   
                      6.0           Western Bulldogs  2018           6.0   
                      7.0           Western Bulldogs  2018           7.0   
                      8.0           Western Bulldogs  2018           8.0   
                      9.0           Western Bulldogs  2018           9.0   
                      10.0          Western Bulldogs  2018          10.0   
                      11.0          Western Bulldogs  2018          11.0   
                      13.0          Western Bulldogs  2018          13.0   
                    